In [15]:
import pickle as pkl
import tensorflow as tf

from keras import models, layers
from keras.src.applications import VGG16
from keras.src.optimizers import Adam

import datasets

In [16]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [17]:
IMG_HEIGHT = 2245
IMG_WIDTH = 1692
IMG_HEIGHT_REDUCED = int(IMG_HEIGHT / 2)
IMG_WIDTH_REDUCED = int(IMG_WIDTH / 2)
CHANNELS = 3
CLASSES = 9

In [18]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(IMG_HEIGHT_REDUCED, IMG_WIDTH_REDUCED, CHANNELS))

for layer in base_model.layers:
    layer.trainable = True
    
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1122, 846, 3)]    0         
                                                                 
 block1_conv1 (Conv2D)       (None, 1122, 846, 64)     1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 1122, 846, 64)     36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 561, 423, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 561, 423, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 561, 423, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 280, 211, 128)     0     

In [19]:
model = models.Sequential()

model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu', name='dense'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(CLASSES, activation='softmax', name='predictions'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 35, 26, 512)       14714688  
                                                                 
 flatten_2 (Flatten)         (None, 465920)            0         
                                                                 
 dense (Dense)               (None, 128)               59637888  
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 predictions (Dense)         (None, 9)                 1161      
                                                                 
Total params: 74353737 (283.64 MB)
Trainable params: 74353737 (283.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
# Load the datasets
training, validation = datasets.get_datasets()

Found 107394 files belonging to 9 classes.
Using 85916 files for training.
Using 21478 files for validation.


In [21]:
model.compile(
    optimizer=Adam(learning_rate=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    x=training,
    epochs=5,
    verbose=1,
    validation_data=validation
).history

Epoch 1/5


2024-03-28 15:00:58.312010: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-28 15:00:58.467516: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


10740/10740 [==============================] - 10987s 1s/step - loss: 0.0581 - accuracy: 0.9877 - val_loss: 0.0665 - val_accuracy: 0.9933
Epoch 2/5
10740/10740 [==============================] - 10919s 1s/step - loss: 0.0207 - accuracy: 0.9954 - val_loss: 0.0222 - val_accuracy: 0.9949
Epoch 3/5
10740/10740 [==============================] - 10925s 1s/step - loss: 0.0148 - accuracy: 0.9966 - val_loss: 0.0207 - val_accuracy: 0.9958
Epoch 4/5
10740/10740 [==============================] - 10917s 1s/step - loss: 0.0124 - accuracy: 0.9971 - val_loss: 0.0239 - val_accuracy: 0.9963
Epoch 5/5
10740/10740 [==============================] - 10904s 1s/step - loss: 0.0108 - accuracy: 0.9975 - val_loss: 0.0206 - val_accuracy: 0.9955


In [22]:
with open('history.pkl', 'wb') as f:
    pkl.dump(history, f)
model.save('model.h5')

/home/tobi/PycharmProjects/Validas/SebastianStrobelHauptprojekt/.venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
loss, accuracy = model.evaluate(validation)
print(f'Loss: {loss}, Accuracy: {accuracy}')

2685/2685 [==============================] - 751s 280ms/step - loss: 0.0206 - accuracy: 0.9955
Loss: 0.02056196704506874, Accuracy: 0.9955303072929382
